# Analyse der Relevanz zwischen Nachrichten und Suchtrends anhand Google-Trends

## Stufe 1: Importierung von den zu nutzenden Bibliotheken und Speicherung der Rohdaten

Es werden die Bibliotheken importiert, die in den sämtlichen Analysenschritten global verwendet werden. Im Anschluss dazu werden die angesammelten Rohdaten durch die API-Anfrage (News-Scraper) aufbereitet und in den folgenden Schritten verarbeitet.

<p><strong>* Importierung </strong></p>

In [64]:
import requests # zuständig für die API-Abfrage
import json # Um den (String-)Content der API-Antworten in JSON-Format umzuwandeln
import csv
import datetime

<p><strong>* Abfrage zur Speicherung der Rohdaten </strong></p>

In [66]:
#Konstanten
VALUE = "value"

URL_FORMAT = "http://h2920750.stratoserver.net:5000/data/{}/all"
NEWS = "news"
TRENDS = "trends"
LIST_ENTITIES = [NEWS, TRENDS]

# Treands-Daten werden täglich um 0 Uhr gecrawlt, weshalb deren Datum um einen Tag substrahiert werden soll.
def subtrace_one_day(data):
    DATE = "date"
    DATE_FORMAT = "%d/%m/%Y"
    substracted = datetime.datetime.strptime(data[DATE], DATE_FORMAT) - datetime.timedelta(days=1)
    data[DATE] = substracted.strftime(DATE_FORMAT)
    return data
    
list_data = [] # 0: news; 1: trends

for ENTITY in LIST_ENTITIES:
    response = requests.get(
        URL_FORMAT.format(ENTITY)
    )
    
    if(not response.ok):
        response.raise_for_status()
    
    data = response.json()[VALUE]
    
    if ENTITY is TRENDS:
        data = list(map(subtrace_one_day, data))
    
    list_data.append(
        data
    )
    
    file = open("./data/raw/{}.csv".format(ENTITY), "w")
    dict_writer = csv.DictWriter(file, data[0].keys())
    dict_writer.writeheader()
    dict_writer.writerows(data)
    file.close()

## Stufe 2: 